In [ ]:
import numpy as np
import pandas as pd

In [ ]:
def answer_one():
    fraud_data = pd.read_csv('fraud_data.csv')
    count_frauds = np.bincount(fraud_data[fraud_data.columns[-1]])[1]
    total_obs = fraud_data.shape[0]
    fraud_pct = count_frauds/total_obs
    return fraud_pct
answer_one()

In [ ]:
# Use X_train, X_test, y_train, y_test for all of the following questions
from sklearn.model_selection import train_test_split

df = pd.read_csv('fraud_data.csv')

X = df.iloc[:,:-1]
y = df.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
def answer_two():
    from sklearn.dummy import DummyClassifier
    from sklearn.metrics import recall_score
    dum_clf = DummyClassifier(strategy = 'most_frequent').fit(X_train, y_train)
    acu_dummy = dum_clf.score(X_test, y_test)
    dum_pred = dum_clf.predict(X_test)
    rec_dummy = recall_score(y_test, dum_pred)    
    return acu_dummy, rec_dummy
answer_two()

In [ ]:
def answer_three():
    from sklearn.metrics import recall_score, precision_score
    from sklearn.svm import SVC
    clf_svm = SVC().fit(X_train, y_train)
    svm_predicted = clf_svm.predict(X_test)
    svm_acc = clf_svm.score(X_test, y_test)
    svm_recall = recall_score(y_test, svm_predicted)
    svm_precision = precision_score(y_test, svm_predicted)
    svm_acc, svm_recall, svm_precision
    return svm_acc, svm_recall, svm_precision
answer_three()

In [ ]:
def answer_four():
    from sklearn.metrics import confusion_matrix
    from sklearn.svm import SVC
    # user defined values for parameters
    threshold_value = -220
    param_values = {'C': 1e9, 'gamma': 1e-07}
    clf_svm2 = SVC(C = param_values['C'], gamma = param_values['gamma']).fit(X_train, y_train)
    clf_svm2_scores = clf_svm2.decision_function(X_test)
    clf_svm2_predicted = clf_svm2.predict(X_test[clf_svm2_scores > threshold_value])
    confusion_svm = confusion_matrix(y_test[clf_svm2_scores > threshold_value], clf_svm2_predicted)
    
    return confusion_svm
answer_four()

In [ ]:
def answer_five():
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import precision_recall_curve, roc_curve

    lm = LogisticRegression().fit(X_train, y_train)
    lm_predicted = lm.predict(X_test)
    lm_probs = lm.predict_proba(X_test)
    precision, recall, thresholds = precision_recall_curve(y_test, lm_probs[:, 1])
    fpr, tpr, thresholds = roc_curve(y_test, lm_probs[:, 1])
    recall = recall[precision == 0.75][0] 
    tpr = tpr[np.around(fpr, decimals = 2) == 0.16][0]
    return recall, tpr
answer_five()

In [ ]:
def answer_six():    
    from sklearn.model_selection import GridSearchCV
    from sklearn.linear_model import LogisticRegression

    lr_model = LogisticRegression().fit(X_train, y_train) 
    grid_values = {'penalty': ['l1', 'l2'], 'C': [0.01, 0.1, 1, 10, 100]}
    grid_lr = GridSearchCV(lr_model, param_grid = grid_values, scoring = 'recall')
    grid_lr.fit(X_train, y_train)
    mean_test_scores = grid_lr.cv_results_['mean_test_score']
    array_test_scores = mean_test_scores.reshape(5,2)
    return array_test_scores
answer_six()

In [ ]:
# Use the following function to help visualize results from the grid search
def GridSearch_Heatmap(scores):
    %matplotlib notebook
    import seaborn as sns
    import matplotlib.pyplot as plt
    plt.figure()
    sns.heatmap(scores.reshape(5,2), xticklabels=['l1','l2'], yticklabels=[0.01, 0.1, 1, 10, 100])
    plt.yticks(rotation=0);

GridSearch_Heatmap(answer_six())